# Especialização em Ciência de Dados - PUC-Rio
# Machine Learning
# Projeto completo de Regressão



## 1. Definição do Problema

Para este projeto, investigaremos o dataset Boston House Price, um dataset para problemas de regressão em que todos os atributos são numéricos.

Cada registro no banco de dados descreve um subúrbio ou cidade de Boston. Os dados foram retirados da Área Estatística Metropolitana Padrão de Boston (SMSA) em 1970. Os atributos são definidos da seguinte forma (retirado do UCI Machine Learning Repository):

1. CRIM: Taxa de criminalidade per capita por cidade
2. ZN: Proporção de terrenos residenciais divididos em lotes com mais de 25.000 pés quadrados
3. INDUS: Proporção de acres não comerciais por cidade
4. CHAS: Variável fictícia CHAS Charles River (= 1 se o trecho limita o rio; 0 caso contrário)
5. NOX: Concentração de óxidos nítricos (partes por 10 milhões)
6. RM: número médio de quartos por habitação
7. AGE: proporção de unidades ocupadas construídas antes de 1940
8. DIS: Distâncias ponderadas para cinco centros de emprego em Boston
9. RAD: Índice de acessibilidade às rodovias radiais
10. TAX: Taxa de imposto sobre o valor total da propriedade por 10 mil dólares
11. PTRATIO: Proporção de alunos por professor por cidade
12. B: 1000(Bk - 0.63)^2 onde Bk é a proporção de negros por cidade
13. LSTAT: % do menor status da população
14. MEDV: Valor médio das casas ocupadas pelos proprietários em mil dólares
Podemos ver que os atributos de entrada têm uma mistura de unidades.

## 2. Carga de Dados

In [0]:
# Imports
import numpy
from numpy import arange
from matplotlib import pyplot
from matplotlib import cm
from pandas import read_csv
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

Faça download do dataset no site do repositório do UCI Machine Learning (https://archive.ics.uci.edu/ml/machine-learning-databases/housing/), salvando no diretório de trabalho local com o nome de arquivo housing.csv.

Iremos especificar os nomes abreviados para cada atributo, para podermos referenciá-los claramente depois. Os atributos são delimitados por espaços em branco, em vez de vírgulas neste arquivo, e indicamos isso para ler a função csv () através do argumento delim_whitespace.

In [0]:
# Carga do dataset
filename = 'housing.csv'
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO',
'B', 'LSTAT', 'MEDV']
dataset = read_csv(filename, delim_whitespace=True, names=names)

## 3. Análise de Dados

### 3.1. Estatísticas Descritivas

In [0]:
# dimensões do dataset
print(dataset.shape)

In [0]:
# tipos de cada atributo
print(dataset.dtypes)

Podemos ver que todos os atributos são numéricos, principalmente valores reais (float) e alguns foram interpretados como números inteiros (int).

In [0]:
 # head
print(dataset.head(20))

Podemos confirmar que as escalas dos atributos são muito diferentes devido às diferentes unidades. Utilizaremos algumas transformações mais tarde. Vamos resumir a distribuição de cada atributo.

In [0]:
# sumário estatístico
set_option('precision', 1) 
print(dataset.describe())

Agora temos uma sensação melhor de como os atributos são diferentes. Os valores mínimo e máximo, bem como as médias, variam muito. Provavelmente, obteremos melhores resultados redimensionando os dados de alguma forma. Agora, vamos dar uma olhada na correlação entre todos os atributos numéricos.

In [0]:
# correlação
set_option('precision', 2) 
print(dataset.corr(method='pearson'))

Podemos ver que muitos dos atributos têm uma forte correlação (por exemplo,> 0,70 ou <-0,70). Por exemplo:
* NOX e INDUS com 0,77.
* DIS e INDUS com -0,71.
* IMPOSTO e INDUS com 0,72.
* IDADE e NOX com 0,73.
* DIS e NOX com -0,78.
Também parece que o LSTAT tem uma boa correlação negativa com a variável de saída MEDV com um valor de -0,74.


### 3.2. Visualizações Unimodais
Vamos agora gerar visualizações dos atributos individualmente.

Primeiro, vamos gerar os histogramas de cada atributo para ter uma ideia das distribuições de dados. 

In [0]:
# histogramas
dataset.hist(sharex=False, sharey=False, xlabelsize=1, ylabelsize=1, figsize=(12,8))
pyplot.show()

Podemos ver que alguns atributos parecem ter uma distribuição exponencial, como CRIM, ZN, AGE e B. Podemos ver que outros parecem ter uma distribuição bimodal, como RAD e TAX. Vejamos as mesmas distribuições usando gráficos de densidade que as suavizam um pouco.

In [0]:
# density plots
dataset.plot(kind='density', subplots=True, layout=(4,4), sharex=False, legend=False, fontsize=1, figsize=(12,8))
pyplot.show()

Isto acrescente mais evidências à nossa suspeita sobre possíveis distribuições exponenciais e bimodais. Também parece que NOX, RM e LSTAT podem ser distribuições Gaussianas distorcidas, informação que pode ser útil posteriormente nas transformações. Vamos analisar os dados com boxplots de cada atributo.

In [0]:
# boxplots
dataset.plot(kind='box', subplots=True, layout=(4,4), sharex=False, sharey=False, fontsize=8, figsize=(12,8))
pyplot.show()

Isso ajuda a apontar tanto a distorção em muitas distribuições, quanto os dados que parecem ser outliers.

### 3.3. Visualizações Multimodais
Vamos visualizar algumas interações entre variáveis, começando com uma matriz de dispersão (scatter plot matrix).

In [0]:
# scatter plot matrix
scatter_matrix(dataset, figsize=(12,8))
pyplot.show()

Podemos ver que alguns dos atributos com correlação mais alta mostram boa estrutura em seu relacionamento. Vamos visualizar também as correlações entre os atributos.

In [0]:
# Matriz de Correlação
fig = pyplot.figure(figsize=(12,8))
ax = fig.add_subplot(111)
cax = ax.matshow(dataset.corr(), vmin=-1, vmax=1, interpolation='none', cmap=cm.get_cmap('RdBu')) 
fig.colorbar(cax)
ticks = numpy.arange(0,14,1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(names)
ax.set_yticklabels(names)
pyplot.show()

A cor azul escuro mostra correlação positiva, enquanto a cor vermelho escuro mostra correlação negativa. Também podemos ver alguns vermelhos e azuis escuros que sugerem candidatos à remoção para melhorar a precisão dos modelos posteriormente.


### 3.4. Resumo das Ideias
Há muita estrutura neste conjunto de dados. Precisamos pensar nas transformações que poderíamos usar posteriormente para melhor expor a estrutura que, por sua vez, pode melhorar a precisão da modelagem. Até agora, valeria a pena tentar:
* Seleção de recursos e remoção dos atributos mais correlacionados.
* Normalização do conjunto de dados para reduzir o efeito de diferentes escalas.
* Padronização do conjunto de dados para reduzir os efeitos de diferentes distribuições.

Também poderíamos explorar a possibilidade de discretização dos dados, o que pode melhorar a precisão dos algoritmos de árvore de decisão.

## 4. Pré-Processamento dos dados

### 4.1. Conjunto de validação

É uma boa prática usar um conjunto de validação (na literatura também chamado de conjunto de testes), uma amostra dos dados que não será usada para a modelagem, mas somente no fim do projeto para confirmar a precisão do  modelo final. É um teste  que podemos usar para verificar se erramos e para nos dar confiança nas estimativas em dados não vistos. Usaremos 80% do conjunto de dados para modelagem e guardaremos 20% para validação.

In [0]:
# Separação em conjuntos de treino e validação
array = dataset.values
X = array[:,0:13]
Y = array[:,13]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y,
    test_size=validation_size, random_state=seed)

## 5. Modelos de Regressão

### 5.1. Criação e avaliação de modelos: linha base

Não sabemos quais modelos performarão bem neste conjunto de dados. Para testar, usaremos a validação cruzada 10-fold e avaliaremos os modelos usando a métrica de Mean Squared Error (MSE). O MSE dará uma idéia geral de quão erradas todas as previsões são (0 é perfeito).

In [0]:
# Parâmetros
num_folds = 10
seed = 7
scoring = 'neg_mean_squared_error'

Vamos criar uma linha base de desempenho para esse problema e verificar vários modelos diferentes com suas configurações padrão:
* **Algortimos Linear:** Linear Regression (LR), Lasso Regression (LASSO) e ElasticNet (EN).
* **Algortimos Não-lineares :** Classification and Regression Trees (CART), Support Vector Regression (SVR) e k-Nearest Neighbors (KNN).

In [0]:
# Criação dos modelos
models = []
models.append(('LR', LinearRegression())) 
models.append(('LASSO', Lasso())) 
models.append(('EN', ElasticNet())) 
models.append(('KNN', KNeighborsRegressor())) 
models.append(('CART', DecisionTreeRegressor())) 
models.append(('SVR', SVR(gamma="auto")))

Agora vamos comparar os modelos, exibindo o MSE médio e o desvio padrão de cada um:

In [0]:
# Avaliação dos modelos
results = []
names = []
for name, model in models:
  kfold = KFold(n_splits=num_folds, random_state=seed)
  cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

A LR tem o MSE mais baixo, seguido de perto pela CART, mas estes são apenas valores médios de MSE, sendo também prudente observar a distribuição dos resultados de cada fold da validação cruzada. Faremos isso com os whisker plots.

In [0]:
# Comparação dos modelos
fig = pyplot.figure() 
fig.suptitle('Comparação dos modelos') 
ax = fig.add_subplot(111) 
pyplot.boxplot(results) 
ax.set_xticklabels(names) 
pyplot.show()

As diferentes escalas dos dados provavelmente estão prejudicando a habilidade de todos os algoritmos e talvez mais ainda para SVR e KNN. A seguir, repetiremos este processo usando uma cópia padronizada do conjunto de dados de treinamento.

### 5.2. Criação e avaliação de modelos: dados padronizados

Como suspeitamos que as diferentes distribuições dos dados brutos possam impactar negativamente a habilidade de alguns modelos, vamos agora utilizar cópia padronizada do dataset. Os dados serão transformados de modo que cada atributo tenha média 0 e um desvio padrão 1. 

Para evitar o "vazamento de dados" na transformação, vamos usar pipelines que padronizam os dados e constroem o modelo para cada fold de teste de validação cruzada. Dessa forma, podemos obter uma estimativa justa de como cada modelo com dados padronizados pode funcionar com dados não vistos.

In [0]:
# Padronização do dataset
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR',
LinearRegression())])))
pipelines.append(('ScaledLASSO', Pipeline([('Scaler', StandardScaler()),('LASSO',
    Lasso())])))
pipelines.append(('ScaledEN', Pipeline([('Scaler', StandardScaler()),('EN', ElasticNet())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsRegressor())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeRegressor())])))
pipelines.append(('ScaledSVR', Pipeline([('Scaler', StandardScaler()),('SVR', SVR())]))) 
results = []
names = []
for name, model in pipelines:
  kfold = KFold(n_splits=num_folds, random_state=seed)
  cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

Podemos ver que a padronização afetou o KNN, diminuindo o erro para abaixo dos outros modelos. O erro do SVR também diminuiu bastante, mas não o suficiente para ser um dos melhores modelos.

Vamos analisar estes resultados graficamente:

In [0]:
# Comparação dos modelos com dados padronizados
fig = pyplot.figure()
fig.suptitle('Comparação dos modelos com dados padronizados') 
ax = fig.add_subplot(111) 
pyplot.boxplot(results) 
ax.set_xticklabels(names)
pyplot.show()

### 5.3. Ajuste dos Modelos


#### Ajuste do KNN

Podemos ajustar o número de vizinhos e as métricas de distância para o KNN. Tentaremos todos os valores ímpares de k de 1 a 21 e as métricas de distância euclideana, manhattan e minkowski. Cada valor de k e de distância será avaliado usando a validação cruzada 10-fold no conjunto de dados padronizado.

In [0]:
# Tuning do KNN

scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

k = [1,3,5,7,9,11,13,15,17,19,21]
distancias = ["euclidean", "manhattan", "minkowski"]
param_grid = dict(n_neighbors=k, metric=distancias)

model = KNeighborsRegressor()

kfold = KFold(n_splits=num_folds, random_state=seed)

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold, iid=True)
grid_result = grid.fit(rescaledX, Y_train)

print("Melhor: %f usando %s" % (grid_result.best_score_, grid_result.best_params_)) 
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) com: %r" % (mean, stdev, param))

Vimos que a melhor configuração tem k = 3 e distância de manhattan, fornecendo um erro quadrático médio de -15.076631, o melhor até agora.

## 6. Métodos Ensemble

Outra maneira de melhorar o desempenho dos algoritmos é usar métodos de ensemble. Avaliaremos quatro modelos diferentes, sendo dois métodos de Boosting e dois de Bagging:
* Métodos de Boosting: AdaBoost (AB) e Gradient Boosting (GBM).
* Métodos de Bagging: Random Forests (RF) e Extra Trees (ET).

Usaremos novamente a validação cruzada 10-fold e pipelines que irão padronizar os dados de treino para cada fold.

In [0]:
# Ensembles

ensembles = []
ensembles.append(('ScaledAB', Pipeline([('Scaler', StandardScaler()),('AB', AdaBoostRegressor())])))
ensembles.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()),('GBM', GradientBoostingRegressor())])))
ensembles.append(('ScaledRF', Pipeline([('Scaler', StandardScaler()),('RF', RandomForestRegressor(n_estimators=10))])))
ensembles.append(('ScaledET', Pipeline([('Scaler', StandardScaler()),('ET', ExtraTreesRegressor(n_estimators=10))])))

results = []
names = []

for name, model in ensembles:
  kfold = KFold(n_splits=num_folds, random_state=seed)
  cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

Podemos ver que em geral obtemos melhores pontuações do que nossos algoritmos lineares e não lineares nas seções anteriores.

Vamos comparar os resultados graficamente:

In [0]:
# Comparação de modelos
fig = pyplot.figure()
fig.suptitle('Comparação de modelos Ensemble com dados padronizados') 
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

Os melhores resultados parecem ser dos modelos Gradient Boosting e Extra Trees. Provavelmente, podemos fazer melhor, já que as técnicas de ensemble usaram os parâmetros padrão. Em seguida, analisaremos o ajuste do Gradient Boosting para aumentar ainda mais o desempenho.

### 6.1. Ajuste dos métodos Ensemble

O número padrão de estimadores é 100 e, frequentemente, quanto maior o número de estimadores, melhor o desempenho, porém maior o tempo de treinamento. Examinaremos o ajuste do número de estimadores, definindo valores entre 50 e 400, em incrementos de 50. Cada configuração será avaliada usando a validação cruzada 10-fold.

In [0]:
# Ajuste do GBM com dados padronizados

scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

param_grid = dict(n_estimators=numpy.array([50,100,150,200,250,300,350,400]))
model = GradientBoostingRegressor(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed)

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold, iid=True)
grid_result = grid.fit(rescaledX, Y_train)
print("Melhor: %f usando %s" % (grid_result.best_score_, grid_result.best_params_)) 

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) com: %r" % (mean, stdev, param))

Podemos ver que a melhor configuração foi com n_estimadores = 400, resultando em um erro quadrado médio de -9,353870, melhor que o método não ajustado.

## 7. Finalização do Modelo

Neste estudo, o gradient boosting apresentou os melhores resultados. Finalizaremos o modelo treinando-o em todo o conjunto de dados de treinamento e faremos predições para o conjunto de dados de validação (separado anteriormente) para confirmar nossas descobertas. 

Iremos padronizar todo o conjunto de dados de treinamento e depois, aplicar a mesma transformação aos atributos de entrada do conjunto de dados de validação.

In [0]:
# Preparação do modelo
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model = GradientBoostingRegressor(random_state=seed, n_estimators=400)
model.fit(rescaledX, Y_train)

# Estimativa do MSE no conjunto de validação
rescaledValidationX = scaler.transform(X_validation)
predictions = model.predict(rescaledValidationX)
print(mean_squared_error(Y_validation, predictions))

Podemos ver que o erro quadrático médio estimado é de 11,8, próximo à nossa estimativa anterior de -9,3.

## Resumo

Trabalhamos com um problema de aprendizado de máquina de modelagem preditiva de regressão de ponta a ponta. As etapas abordadas foram:

* Definição do problema (Boston house price data).
* Carga dos dados.
* Análise dos dados (algumas distribuições distorcidas e atributos correlatos).
* Avaliação de modelos (Linear Regression parecia bom).
* Avaliação de modelos com padronização (KNN parecia bom).
* Ajuste dos modelos (K=3 para KNN foi melhor).
* Métodos de ensemble (Bagging and Boosting, Gradient Boosting parecia bom).
* Ajuste dos métodos de Ensemble (melhorando o Gradient Boosting).
* Finalização do modelo (use todos os dados de treinamento e valide usando o conjunto de dados de validação).